## PROJET HADOOP - MS-SIO-2019 - SNCF - API TRANSILIEN - PARTIE I & II

#### SPARK STRUCTURED STREAMING (KAFKA CONSUMER)

P. Hamy,  N. Leclercq, L. Poncet - MS-SIO-2019

In [1]:
import os
import json
import time
import logging
from pyspark.sql import SparkSession
import pyspark.sql.types as st
import pyspark.sql.functions as sf

In [2]:
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.ERROR, datefmt='%H:%M:%S')

Changement du logging level afin d'éliminer le bruit généré dans la console par un [_warning_](https://stackoverflow.com/questions/39351690/got-interruptedexception-while-executing-word-count-mapreduce-job) récurrent

In [3]:
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

### QUESTION 1.1 : calcul et publication du temps d'attente moyen par station

Cette section donne le détail de construction du flux de données relatif à la dernière heure - tranche horaire à laquelle les métriques demandées s'appliquent. Le code est segmenté afin d'en faliciter le commentaire. Il sera repris plus loin afin d'être encapsulé dans une classe.  

#### Création de la session Spark associé au flux Kafka

In [4]:
kafka_session = SparkSession.builder.appName("MS-SIO-HADOOP-PROJECT-KAFKA-STREAM").getOrCreate()

Limitation du nombre de taches lancées par spark (conseil de configutation glané sur internet pour les configurations matérielles les plus modestes).

In [5]:
kafka_session.conf.set('spark.sql.shuffle.partitions', 4)

#### Création du flux Kafka
On utilise ici un [structured spark stream](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html) associé à une source Kafka. 

Il s'agit de spécifier la source via l'adresse du serveur Kafka et le nom du topic auquel on souhaite s'abonner. 

In [6]:
kafka_stream = kafka_session \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "sandbox-hdp.hortonworks.com:6667") \
    .option("subscribe", "transilien-02") \
    .option("startingOffsets", "earliest") \
    .option("kafkaConsumer.pollTimeoutMs", 512) \
    .load()

#### Schéma de désérialisation des messages  
Les messages injectés dans le flux Kafka sont sérialisés et encodés en binaire dans le champ _value_ du dataframe (format générique des dataframe issus d'un stream Kafka).
```
kafka_stream.printSchema()
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 ```
Il est donc nécessaire despécifier le schéma de désérialisation qui sera passé à la fonction **from_json**.

In [7]:
json_schema = st.StructType(
    [
        st.StructField("station", st.IntegerType(), True),
        st.StructField("train", st.StringType(), True),
        st.StructField("timestamp", st.TimestampType(), True),
        st.StructField("mode", st.StringType(), True),
        st.StructField("mission", st.StringType(), True),
        st.StructField("terminus", st.IntegerType(), True)
    ]
)

A travers, la variable **json_options**, on précise également le format du champ _timestamp_ afin que les valeurs temporelles soient correctement interprétées.

In [ ]:
json_options = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"}

#### Séquence de calcul du temps d'attente moyen par station : étape-01 
Désérialisation/reformatage des messages.

In [ ]:
df = kafka_stream \
    .select(sf.from_json(sf.col("value").cast("string"), json_schema, json_options).alias("departure")) \
    .select("departure.*")

A l'issue de opération le dataframe a le schéma suivant:
```
df.printSchema()
root
 |-- station: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- train: string (nullable = true)
```
#### Séquence de calcul du temps d'attente moyen par station : étape-02
Spécification de la watermark du stream spark. Nous n'acceptons pas les messages ayant plus d'une minute de retard. Il s'agit d'un choix arbitraire qui n'a que peu d'intérêt dans notre projet. Il est toutefois nécessaire de spécifier cette valeur car l'implémentation sous-jacente doit borner l'accumulation des données du stream. [La documentation de Spark explique clairement le concept de _Stateful Stream Processing in Structured Streaming_](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html).

In [ ]:
df = df.withWatermark("timestamp", "1 minutes")

#### Séquence de calcul du temps d'attente moyen par station : étape-03
Un train apparaitra dans les réponses aux requêtes de l'API SNCF tant que son heure de départ n'appartient pas au passé. On supprime donc les doublons associés aux couples (train, heure de départ). Inutile d'ajouter la station à la contrainte d'exclusion car l'idenfiant d'un train est unique.

In [ ]:
df = df.dropDuplicates(["train", "timestamp"])

#### Séquence de calcul du temps d'attente moyen par station : étape-04
Définition de la fênêtre (temporelle) dans laquelle les calculs sont aggrégés. Il s'agit ici de calculer le temps d'attente moyen par station **sur la dernière heure**. Notre fenêtre a donc une largeur temporelle de 60 minutes (_window length_). On choisit de suivre cette moyenne par pas de 2 minutes (_sliding interval_). Dans la mesure où le calcul est demandé par station, la fonction _groupBy_ s'applique au champ _station_ du dataframe.

In [ ]:
df = df.groupBy("station", sf.window("timestamp", "60 minutes", "2 minutes"))

#### Séquence de calcul du temps d'attente moyen par station : étape-05
Pour chaque station, on définit le temps moyen d'attente sur une période de P minutes comme le rapport de P vs le nombre de trains au départ de cette station sur la période P. Ici P = 60 minutes. 

On crée une _aggrégation_ qui contiendra, pour chaque station et pour chaque fenêtre d'une heure :
- une colonne _nt_ qui indiquant le nombre de trains sur la période
- une colonne _awt_ donnant le temps d'attente moyen recherché. 

La colonne _nt_ est injectée à titre indicatif (visualisation dans la console, validation du calcul).

In [ ]:
df = df.agg(sf.count("train").alias("nt"), sf.format_number(60./ sf.count("train"), 2).cast("double").alias("awt"))

#### Séquence de calcul du temps d'attente moyen par station : étape-06
Selection de la fenêtre temporelle associée à la dernière heure. 

Il s'agit de selectionner, parmis les N fenêtres temporelles produites par Spark, celle qui correspond à la dernière heure écoulée. On utilise ici la fonction **current_timestamp** de Spark afin de rendre la sélection dynamique (i.e. glissante). Le calcul est effectué est dans l'unité de **unix_timestamp** (la seconde) - beaucoup plus facile à manipulée dans ce contexte. 

Dans l'idée de pourvoir visualiser (si besoin) les valeurs mises en jeu dans le calcul, on choisit de créér les colonnes associées:
- oha = one hour ago = now - 62 minutes = now - (window length + sliding interval) 
- now = now - 1 minutes = now - (sliding interval / 2.) => **valeur ajustée pour n'obtenir qu'une seule fenêtre**
- wstart = window.start = borne inférieure de la fenêtre temporelle 
- wend = window.wend = borne supérieure de la fenêtre temporelle

**La clause _where_ permet de selectionner la fenêtre associée à la dernière heure**.  

In [ ]:
df = df \
    .withColumn("oha", sf.unix_timestamp(sf.current_timestamp()) - int((60 + 2) * 60)) \
    .withColumn("now", sf.unix_timestamp(sf.current_timestamp()) - int(60 * 2) / 2.) \
    .withColumn("wstart", sf.unix_timestamp("window.start")) \
    .withColumn("wend", sf.unix_timestamp("window.end")) \
    .where((sf.col("oha") <= sf.col("wstart")) & (sf.col("wend") <= sf.col("now")))

A ce stade, _df_ constitue 'l'état' de référence de notre stream Spark. C'est à partir de cet état que l'on produit les résultats, métriques, indicateurs, ... demandés

#### Validation des étapes 01 à 06

On lance un 'writeStream' vers la console afin de visualiser les données produites par le stream.

In [ ]:
query = df \
    .select("station", "window", "nt", "awt") \
    .writeStream \
    .format("console") \
    .option("truncate", False) \
    .outputMode("complete") \
    .start()

````
+--------+------------------------------------------+---+----+
|station |window                                    |nt |awt |
+--------+------------------------------------------+---+----+
|87382473|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|14 |4.29|
|87386425|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|14 |4.29|
|87382259|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|19 |3.16|
|87382457|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|18 |3.33|
|87382440|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|20 |3.0 |
|87382333|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|14 |4.29|
|87382887|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|12 |5.0 |
|87334482|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|30 |2.0 |
|87381137|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|48 |1.25|
|87386318|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|17 |3.53|
|87382374|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|12 |5.0 |
|87382499|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|27 |2.22|
|87382655|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|5  |12.0|
|87382382|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|25 |2.4 |
|87381905|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|31 |1.94|
|87384008|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|16 |3.75|
|87386003|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|19 |3.16|
|87386300|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|24 |2.5 |
|87381129|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|38 |1.58|
|87386409|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|24 |2.5 |
+--------+------------------------------------------+---+----+
```

Arrêt de la query

In [ ]:
query.stop()

In [ ]:
kafka_session.streams.active

### QUESTION 1.2 à 1.6
- Calcul du temps moyen d’attente sur la ligne station par station sur la dernière heure
- Calcul du temps moyen d’attente globale sur la ligne sur la dernière heure
- Trier les stations par temps d’attente moyen sur la dernière heure
- Trouver la station avec le temps d’attente le plus élevée sur la dernière heure
- Trouver la station avec le temps d’attente le moins élevée sur la dernière heure
- Construire un tableau de bord dans Tableau Software sur la base de ces indicateurs

Les calculs demandés nécessiteraient une seconde opération aggrégation sur le stream (last_hour_stream). Or, en l'état actuel de Spark (2.4), [il n'est pas possible d'enchainer plusieurs opération d'aggrégation sur un même stream](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#unsupported-operations). Il nous faut donc trouver une solution de contournement.

L'idée retenue est d'effectuer les calculs sur chaque batch de last_hour_stream. Cette approche fonctionne ici car chaque batch contient l'intégralité des données sur lesquelles les calculs doivent être réalisés pour produire les résultats attendus - i.e. les données d'attente moyenne par station. Ces résulats sont enregistrés sous _Hive_ dans des tables spécifiques. Ils sont ainsi rendus accéssibles depuis Tableau pour l'élaboration du tableau de bord. 

Les calculs (Q1.2 à Q.1.5) sont regroupés dans un callback du type _foreachBatch_ dont les appels sont déclenchés par une _StreamingQuery_.

La classe _TransilienStreamProcessor_ implémente les fonctionnalités demandées.

In [1]:
import os
import json
import time
import logging
from pyspark.sql import SparkSession
import pyspark.sql.types as st
import pyspark.sql.functions as sf
from py4j.java_gateway import java_import
from api_transilien_tools import NotebookCellContent

In [2]:
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.ERROR, datefmt='%H:%M:%S')

Changement du logging level afin d'éliminer le bruit généré dans la console par un [_warning_](https://stackoverflow.com/questions/39351690/got-interruptedexception-while-executing-word-count-mapreduce-job) récurrent

In [3]:
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

In [4]:
class TransilienStreamProcessor(NotebookCellContent):
    
    # -------------------------------------------------------------------------------
    def __init__(self, config):
    # -------------------------------------------------------------------------------
        NotebookCellContent.__init__(self, "TransilienStreamProcessor")
    
        # store configuration 
        self.config = config
        
        # setup logger
        self.set_logging_level(logging.DEBUG if self.config['verbose'] else logging.ERROR)
        self.debug("TSP:initializing...")
        
        # kafka oriented spark session (configured to process incoming Kafka messages)
        self.debug(f"TSP:creating kafka oriented spark session")
        self.kafka_session = SparkSession \
            .builder \
            .master("yarn") \
            .appName("MS-SIO-HADOOP-PROJECT-STREAM") \
            .config("spark.sql.shuffle.partitions", self.config['spark_sql_shuffle_partitions']) \
            .config('spark.sql.hive.thriftServer.singleSession', True) \
            .config('hive.server2.thrift.port', self.config['hive_thrift_server_port']) \
            .enableHiveSupport() \
            .getOrCreate()
        self.debug(f"`-> done!")
        
        # kafka 'last hour awt' stream initialization
        self.debug(f"TSP:initializing 'last hour awt' stream")
        self.last_hour_awt_stream = self.setup_last_hour_awt_stream()
        self.debug(f"`-> done!")
        
        # kafka 'trains progression' stream initialization
        self.debug(f"TSP:initializing 'trains progression' stream")
        self.trains_progression_stream = self.setup_trains_progression_stream()
        self.debug(f"`-> done!")
        
        # start our own thrift server (in singleSession mode) 
        # this will allow to expose the temp. views and make them reachable from Tableau Software   
        self.__start_thrift_server()
            
        # create a temp. view for the transilien stations data (label, geo.loc., ...)
        self.__create_stations_view()
        
        # the last hour awt hive sink (streaming query)
        # acts as a trigger for computeAwtMetricsAndSaveAsTempViews (forEachBatch callback)
        self.lhawt_hive_sink = None
        
        # the console sibk (streaming query) 
        # print batches into the console  
        self.lhawt_console_sink = None
        
        # the trains progression hive sink (streaming query)
        # acts as a trigger for computeTrainsProgressionAndSaveAsTempView (forEachBatch callback)
        self.trprg_hive_sink = None
        
        # processing time (i.e. streaming queries trigger period)
        self.processing_time = "10 seconds"
        
        # start the streaming queries?
        if self.config['auto_start']:
            self.start()
            
        self.debug(f"initialization done!")
        
    # -------------------------------------------------------------------------------
    def __start_thrift_server(self):
    # -------------------------------------------------------------------------------
        try:
            self.debug(f"TSP:starting thrift server on port {self.config['hive_thrift_server_port']}") 
            #sc.setLogLevel('INFO')
            java_import(sc._gateway.jvm,"")
            sc._gateway.jvm.org.apache.spark.sql.hive.thriftserver.HiveThriftServer2.startWithContext(spark._jwrapped)
            #sc.setLogLevel('ERROR')       
            self.debug(f"TSP:thrift server successfully started") 
        except Exception as e:
            self.error(e)
            
    # -------------------------------------------------------------------------------
    def __create_stations_view(self):
    # -------------------------------------------------------------------------------
        # read data local data file       
        df = self.kafka_session \
            .read \
            .format("csv") \
            .option("sep", ",") \
            .option("inferSchema", "true") \
            .option("header", "true") \
            .load("file:/root/ms-sio-hdp/api-transilien/transilien_line_l_stations_by_code.csv") \
            .createOrReplaceTempView("stations_data")
                   
    # -------------------------------------------------------------------------------
    def setup_last_hour_awt_stream(self):
    # -------------------------------------------------------------------------------
        wm = float(self.config['kafka_lhawt_stream_watermark'])
        wl = float(self.config['kafka_lhawt_stream_window_length'])
        si = float(self.config['kafka_lhawt_stream_sliding_interval'])
        oha_offset = int((wl + si) * 60.)
        now_offset = int(60. * si / 2.)
        # setup 'last hour stream' (see above for details)
        return self.kafka_session \
            .readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", self.config['kafka_broker']) \
            .option("subscribe", self.config['kafka_topic']) \
            .option("startingOffsets", "earliest") \
            .load() \
            .select(sf.from_json(sf.col("value").cast("string"), self.config['json_schema'], self.config['json_options']).alias("departure")) \
            .select("departure.*") \
            .withWatermark("timestamp", f"{int(wm)} minutes") \
            .dropDuplicates(["train", "timestamp"]) \
            .groupBy("station", sf.window("timestamp", f"{int(wl)} minutes", f"{int(si)} minutes")) \
            .agg(sf.count("train").alias("nt"), sf.format_number(wl / sf.count("train"), 1).cast("double").alias("awt")) \
            .withColumn("oha", sf.unix_timestamp(sf.current_timestamp()) - oha_offset) \
            .withColumn("now", sf.unix_timestamp(sf.current_timestamp()) - now_offset) \
            .withColumn("wstart", sf.unix_timestamp("window.start")) \
            .withColumn("wend", sf.unix_timestamp("window.end")) \
            .where((sf.col("oha") <= sf.col("wstart")) & (sf.col("wend") <= sf.col("now"))) \
            .drop("oha", "now", "wstart", "wend")
      
    # -------------------------------------------------------------------------------
    def setup_trains_progression_stream(self):
    # -------------------------------------------------------------------------------
        time_window = config['kafka_trprg_time_window']
        contiguous_stations = self.config['kafka_trprg_stream_stations']
        return self.kafka_session \
            .readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "sandbox-hdp.hortonworks.com:6667") \
            .option("subscribe", "transilien-02") \
            .option("startingOffsets", "earliest") \
            .load() \
            .select(sf.from_json(sf.col("value").cast("string"), self.config['json_schema'], self.config['json_options']).alias("departure")) \
            .select("departure.*") \
            .filter(sf.col("station").isin(list(contiguous_stations.values()))).filter("mode='R'") \
            .withColumn("departure", sf.unix_timestamp("timestamp")).drop("timestamp") \
            .where(sf.col("departure").between(sf.unix_timestamp(sf.current_timestamp()) - int(time_window/2.), \
                                               sf.unix_timestamp(sf.current_timestamp()) + int(time_window/2.))) \
            .groupBy("train", "station", "departure", "mode", "mission", "terminus").agg(sf.count("train").alias("tmp")).drop("tmp")
  
    # -------------------------------------------------------------------------------
    def start(self):
    # -------------------------------------------------------------------------------
        self.start_last_hour_awt_stream()
        self.start_trains_progress_stream()
     
    # -------------------------------------------------------------------------------
    def stop(self):
    # -------------------------------------------------------------------------------
        self.stop_last_hour_awt_stream()
        self.stop_trains_progress_stream()
                       
    # -------------------------------------------------------------------------------
    def start_last_hour_awt_stream(self):
    # -------------------------------------------------------------------------------
        # stop the streaming queries if already running
        self.stop_last_hour_awt_stream()
                       
        # last hour awt: create then start the hive sink (streaming query)
        # also make 'self.computeAwtMetricsAndSaveAsTempViews' member function the associated 'foreachBatch' callback
        self.debug(f"TSP:starting hive sink for the 'last hour average waiting time' stream (streaming query)")
        self.lhawt_hive_sink =  self.last_hour_awt_stream \
                            .writeStream \
                            .trigger(processingTime=self.processing_time) \
                            .foreachBatch(self.computeAwtMetricsAndSaveAsTempViews) \
                            .outputMode("complete") \
                            .start()
        self.debug(f"`-> done!")
                       
        # last hour awt: create then start the console sink (streaming query)
        self.debug(f"TSP:starting console sink for the 'last hour average waiting time' stream (streaming query)")
        self.lhawt_console_sink = self.last_hour_awt_stream \
                            .orderBy("awt") \
                            .writeStream \
                            .trigger(processingTime=self.processing_time) \
                            .outputMode("complete") \
                            .format("console") \
                            .option("truncate", False) \
                            .start() 
        self.debug(f"`-> done!")
                       
    # -------------------------------------------------------------------------------
    def start_trains_progress_stream(self):
    # -------------------------------------------------------------------------------
        # stop the streaming queries if already running
        self.stop_trains_progress_stream()
                       
        # trains progression: create then start the hive sink (streaming query)
        # also make 'self.computeTrainsProgressionAndSaveAsTempView' member function the associated 'foreachBatch' callback
        self.debug(f"TSP:starting console sink for the 'trains progression' stream (streaming query)")
        self.trprg_hive_sink = self.trains_progression_stream \
                            .writeStream \
                            .foreachBatch(self.computeTrainsProgressionAndSaveAsTempView) \
                            .outputMode("complete") \
                            .start()
        self.debug(f"`-> done!")
        self.debug(f"TSP:streaming queries are running")
                       
    # -------------------------------------------------------------------------------
    def stop_last_hour_awt_stream(self):
    # -------------------------------------------------------------------------------
        # stop the streaming queries (best effort impl.)
        if self.lhawt_hive_sink is  not None:
            try:
                self.debug(f"TSP:stopping hive sink for the 'last hour average waiting time' stream (streaming query)")
                self.lhawt_hive_sink.stop()
            except Exception as e:
                pass
            finally:
                self.lhawt_hive_sink = None
                self.debug(f"`-> done!")
        if self.lhawt_console_sink is  not None:
            try:
                self.debug(f"TSP:stopping console sink for the 'last hour average waiting time' stream (streaming query)")
                self.lhawt_console_sink.stop()
            except Exception as e:
                pass
            finally:
                self.lhawt_console_sink = None
                self.debug(f"`-> done!")
   
    # -------------------------------------------------------------------------------
    def stop_trains_progress_stream(self):
    # -------------------------------------------------------------------------------
        # stop the streaming queries (best effort impl.)
        if self.trprg_hive_sink is  not None:
            try:
                self.debug(f"TSP:stopping hive sink for the 'trains progression stream' (streaming query)")
                self.trprg_hive_sink.stop()
            except Exception as e:
                pass
            finally:
                self.trprg_hive_sink = None
                self.debug(f"`-> done!")
                       
    # -------------------------------------------------------------------------------
    def cleanup(self):
    # -------------------------------------------------------------------------------
        # cleanup the underlying session 
        # TODO: not sure this is the right way to do the job
        self.debug(f"TSP:shutting down Kafka-SparkSession")
        self.kafka_session.stop()
        self.debug(f"`-> done!")
        self.debug(f"TSP:shutting down Hive-SparkSession")
        self.hive_session.sql("clear cache")
        self.hive_session.stop()
        self.debug(f"`-> done!")
      
    # -------------------------------------------------------------------------------
    def turnVerboseOn(self):
    # -------------------------------------------------------------------------------
        # turn verbose on
        self.set_logging_level(logging.DEBUG)
       
    # -------------------------------------------------------------------------------
    def turnVerboseOff(self):
    # -------------------------------------------------------------------------------
        # turn verbose off
        self.set_logging_level(logging.ERROR)
        
    # -------------------------------------------------------------------------------
    def computeAwtMetricsAndSaveAsTempViews(self, batch, batch_number):
    # -------------------------------------------------------------------------------
        # PART-I: COMPUTE AVERAGE WAITING TIME METRICS
        # --------------------------------------------
        # this 'forEachBatch' callback is attached to the our 'lhawt_hive_sink' (streaming query)
        try:
            # clear cell content so that we don't cumulate the log               
            #self.clear_output()
                              
            # be sure we have some data to handle (incoming dataframe not empty)
            # this will avoid creating empty tables on Hive side 
            if batch.rdd.isEmpty():
                self.warning(f"TSP:ignoring empty batch #{batch_number}")
                return

            self.debug(f"TSP:entering computeAwtMetricsAndSaveAsTempViews for batch #{batch_number}...")
                              
            # PART-I: Q1.1 & Q1.3: compute ordered average waiting time in minutes (on the last hour period)
            self.debug(f"computing ordered average waiting time...")
            t = time.time()
            tmp = batch.orderBy(sf.asc("awt")).select(batch.station, batch.awt)    
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("ordered_awt")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
                                                                  
            # PART-I: Q1.2: compute global average waiting time in minutes (on the last hour period) 
            self.debug(f"computing global average waiting time...")
            t = time.time()
            tmp = batch.agg(sf.count("station").alias("number_of_stations"), sf.avg("awt").alias("global_awt"))
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("global_awt")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
            
            # PART-I: Q1.4: compute min average waiting time in minutes (on the last hour period)
            self.debug(f"computing min. average waiting time...")
            t = time.time()
            tmp = batch.orderBy(sf.asc("awt")).limit(1).select(batch.station, batch.awt.alias("min_awt"))
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("min_awt")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
           
            # PART-I: Q1.5: compute max average waiting time in minutes (on the last hour period)
            self.debug(f"computing min. average waiting time...")
            t = time.time()
            tmp = batch.orderBy(sf.desc("awt")).limit(1).select(batch.station, batch.awt.alias("max_awt"))
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("max_awt")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
                              
            self.debug(f"TSP:computeAwtMetricsAndSaveAsTempViews successfully executed for batch #{batch_number}")
        except Exception as e:
            self.error(f"TSP:failed to update Hive tables for batch #{batch_number}")
            self.error(e)
                       
    # -------------------------------------------------------------------------------    
    def computeTrainsProgressionAndSaveAsTempView(self, batch, batch_number):
    # -------------------------------------------------------------------------------
        # PART-II: COMPUTE TRAINS PROGRESSION
        # ------------------------------------
        # this 'forEachBatch' callback is attached to the our 'trprg_hive_sink' (streaming query)
        try:
                       
            # clear cell content so that we don't cumulate the log               
            self.clear_output()
                              
            # be sure we have some data to handle (incoming dataframe not empty)
            # this will avoid creating empty tables on Hive side 
            if batch.rdd.isEmpty():
                self.warning(f"TSP:ignoring empty batch #{batch_number}")
                return

            self.debug(f"TSP:entering computeTrainsProgressionAndSaveAsTempView for batch #{batch_number}...")
  
            t = time.time()
            contiguous_stations = self.config['kafka_trprg_stream_stations']
            
            # the main trick: covert rows to columns using spark SQL Pivot
            # https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=pivot
            tmp = batch.groupby("train").pivot("station").max("departure").fillna(0)
            # rename columns: station-a code => sa & station-b code => sb      
            tmp = tmp.select("train", sf.col(str(contiguous_stations['sa'])).alias("sa"), sf.col(str(contiguous_stations['sb'])).alias("sb"))
            # remove undefined columns  (i.e. the ones containing the 'fillna' value - see above)
            tmp = tmp.filter((tmp.sa > 0) & (tmp.sb > 0))
            # compute delta time between contiguous stations
            tmp = tmp.withColumn("dt", tmp.sb - tmp.sa)
            # compute travel direction: 1 for a -> b : 1 or -1 for b -> a (column created for debugging purpose)   
            #tmp = tmp.withColumn("direction", sf.when(sf.col("dt") > sf.lit(0.), sf.lit(1)).otherwise(sf.lit(-1)))
            # insert a column containing the 'now' timestamp 
            tmp = tmp.withColumn("now", sf.unix_timestamp(sf.current_timestamp()))
            # is the train travel between station-a & station-b (or station-b to station-a) belongs to the past? (column created for debugging purpose)
            #tmp = tmp.withColumn("in_past", (tmp.now > tmp.sa) & (tmp.now > tmp.sb))
            # is the train travel between station-a & station-b (or station-b to station-a) belongs to the future? (column created for debugging purpose)
            #tmp = tmp.withColumn("in_future", (tmp.now < tmp.sa) & (tmp.now < tmp.sb))
            # is the train travel between station-a & station-b (or station-b to station-a) in progress?
            tmp = tmp.withColumn("in_progress", (~((tmp.now > tmp.sa) & (tmp.now > tmp.sb))) & (~((tmp.now < tmp.sa) & (tmp.now < tmp.sb))))
            #tmp = tmp.filter(sf.col("in-progress") | sf.col("in-future"))
            # select the right departure time: depends on the travel direction (A->B or B->A)
            tmp = tmp.withColumn("departure_uxts", sf.when(tmp.dt > sf.lit(0), tmp.sa).otherwise(tmp.sb))
            # convert departure time to humanly readable format      
            tmp = tmp.withColumn("departure_date", sf.from_unixtime(sf.col("departure_uxts")))
            # order by departure time
            #tmp = tmp.orderBy(sf.col("departure_uxts"))
            # log (for debugging purpose)
            #tmp.show()
            # compute trains progression: (now - depature_time) / (travel time between stations a & b)
            #tmp = tmp.withColumn("progress", sf.format_number((100. * (tmp.now - tmp.departure_uxts)) / sf.abs(tmp.dt), 1).cast("double"))  
            tmp = tmp.withColumn("progress", (100. * (tmp.now - tmp.departure_uxts)) / sf.abs(tmp.dt))  
            # compute trains progression: maintain value in  the [O, 100]% range 
            tmp = tmp.withColumn("progress", sf.when(tmp.progress < sf.lit(0.), sf.lit(0.)).otherwise(tmp.progress))             
            # compute trains progression: maintain value in  the [O, 100]% range 
            tmp = tmp.withColumn("progress", sf.when(tmp.progress > sf.lit(100.), sf.lit(100.)).otherwise(tmp.progress))
            # compute progress bar value than will be displayed in Tableau Software (this is a trick to display travel direction)            
            tmp = tmp.withColumn("progress_bar_value", sf.when(tmp.in_progress, sf.when(tmp.dt > sf.lit(0.), tmp.progress).otherwise(100. - tmp.progress)).otherwise(tmp.progress))
            # round progress values to 1 digit
            tmp = tmp.withColumn("progress", sf.format_number(tmp.progress, 1).cast("double"))
            tmp = tmp.withColumn("progress_bar_value", sf.format_number(tmp.progress_bar_value, 1).cast("double"))
            # re-inject columns from intial batch (some info could be useful to display in Tableau)
            tmp = tmp.join(batch, "train", how="left")
            # remove duplicates 
            tmp = tmp.dropDuplicates(["train"])
            # log (for debugging purpose)
            # tmp.show()
            # order by departure time
            tmp = tmp.orderBy(tmp.departure_uxts)
            # select required columns
            #tmp = tmp.select("train", sf.col("departure_date").alias("departure"), "station", "mission", "progress", "direction", "progress_bar_value", "in_past", "in_future", "in_progress")
            tmp = tmp.select("train", sf.col("departure_date").alias("departure"), "station", "mission", "progress", "progress_bar_value", "in_progress")
            # log (for debugging purpose)
            tmp.show()
                                 
            # create a temp. view that - visible from Tableau             
            self.kafka_session.createDataFrame(tmp.rdd).createOrReplaceTempView("trains_progression")
            self.debug(f"`-> took {round(time.time() - t, 2)} s")
                              
            self.debug(f"TSP:computeTrainsProgressionAndSaveAsTempView successfully executed for batch #{batch_number}")
        except Exception as e:
            self.error(f"TSP:failed to update Hive tables for batch #{batch_number}")
            self.error(e)

Pour mémoire : dans un contexte de production, le code qui précède serait injecté dans un script Python lancé via _spark-submit_. Dans un tel cas, serait nécessaire placer un appel à _awaitTermination_ sur chaque streaming query.

In [5]:
# configuration parameters
config = {}

# json schema & options for kafka messages deserialization 
config['json_schema'] = st.StructType(
    [
        st.StructField("station", st.IntegerType(), True),
        st.StructField("train", st.StringType(), True),
        st.StructField("timestamp", st.TimestampType(), True),
        st.StructField("mode", st.StringType(), True),
        st.StructField("mission", st.StringType(), True),
        st.StructField("terminus", st.IntegerType(), True)
    ]
)
config['json_options'] = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"}

# spark sesssions options
config['spark_sql_shuffle_partitions'] = 4

# kafka source configuration: broker & topic
config['kafka_broker'] = "sandbox-hdp.hortonworks.com:6667"
config['kafka_topic'] = "transilien-02"

# kafka stream configuration: structured stream windowing for the last hour average waiting time stream
config['kafka_lhawt_stream_watermark'] = 1 
config['kafka_lhawt_stream_window_length'] = 60
config['kafka_lhawt_stream_sliding_interval'] = 2

# kafka stream configuration: contiguous stations for the trains progression stream
config['kafka_trprg_stream_stations'] = {
    'sa':87381129, # Station A: CLICHY LEVALLOIS
    'sb':87382002  # Station B: BECON LES BRUYERES
    
    #'sb':87381137  # Station B: ASNIERES SUR SEINE
    
}

# kafka stream configuration: time window for the trains progression stream
config['kafka_trprg_time_window'] = 1800

# hive (local) thrift server configuration
config['hive_thrift_server_port'] = 10015

# misc. options
config['auto_start'] = True 
config['verbose'] = True

In [6]:
ti = TransilienStreamProcessor(config)

17:05:47 DEBUG:TSP:entering computeTrainsProgressionAndSaveAsTempView for batch #0...
17:05:49 DEBUG:TSP:entering computeAwtMetricsAndSaveAsTempViews for batch #0...
17:05:49 DEBUG:computing ordered average waiting time...
17:05:52 DEBUG:`-> took 2.34 s
17:05:52 DEBUG:computing global average waiting time...
17:05:53 DEBUG:`-> took 0.97 s
17:05:53 DEBUG:computing min. average waiting time...
17:05:54 DEBUG:`-> took 0.91 s
17:05:54 DEBUG:computing min. average waiting time...
17:05:55 DEBUG:`-> took 0.99 s
17:05:55 DEBUG:TSP:computeAwtMetricsAndSaveAsTempViews successfully executed for batch #0


+------+-------------------+--------+-------+--------+------------------+-----------+
| train|          departure| station|mission|progress|progress_bar_value|in_progress|
+------+-------------------+--------+-------+--------+------------------+-----------+
|133613|2019-03-03 16:55:00|87382002|   VOLA|   100.0|             100.0|      false|
|133672|2019-03-03 17:01:00|87382002|   POVA|    97.3|               2.7|       true|
|133617|2019-03-03 17:10:00|87382002|   VOLA|     0.0|               0.0|      false|
|133676|2019-03-03 17:15:00|87382002|   POVA|     0.0|               0.0|      false|
+------+-------------------+--------+-------+--------+------------------+-----------+



17:05:59 DEBUG:TSP:entering computeAwtMetricsAndSaveAsTempViews for batch #1...
17:05:59 DEBUG:computing ordered average waiting time...
17:06:00 DEBUG:`-> took 13.79 s
17:06:00 DEBUG:TSP:computeTrainsProgressionAndSaveAsTempView successfully executed for batch #0
17:06:01 DEBUG:`-> took 1.67 s
17:06:01 DEBUG:computing global average waiting time...
17:06:01 DEBUG:`-> took 0.52 s
17:06:01 DEBUG:computing min. average waiting time...
17:06:02 DEBUG:`-> took 0.47 s
17:06:02 DEBUG:computing min. average waiting time...
17:06:02 DEBUG:`-> took 0.48 s
17:06:02 DEBUG:TSP:computeAwtMetricsAndSaveAsTempViews successfully executed for batch #1
17:08:06 DEBUG:TSP:stopping hive sink for the 'last hour average waiting time' stream (streaming query)
17:08:06 DEBUG:`-> done!
17:08:06 DEBUG:TSP:stopping console sink for the 'last hour average waiting time' stream (streaming query)
17:08:06 DEBUG:`-> done!
17:08:06 DEBUG:TSP:stopping hive sink for the 'trains progression stream' (streaming query)
17:0

In [ ]:
ti.turnVerboseOff()

In [ ]:
ti.turnVerboseOn()

In [7]:
ti.stop()

cleanup...

NB: the python kernel must be restart after a call to TransilienStreamProcessor.cleanup 

In [ ]:
ti.cleanup()